# Meridonal Overturning Circulation
- calculations may be improved by taking partial bottom cells into account
- possibly SSH adds accuracy, but it is centered on the T-grid

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import cmocean
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from MOC import calculate_MOC, AMOC_max, approx_lats
from grid import create_tdepth
from regions import Atlantic_mask, boolean_mask
from paths import file_ex_ocn_ctrl, path_results
from plotting import discrete_cmap, shifted_color_map
from timeseries import IterateOutputCESM, lowpass

In [ ]:
AMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/AMOC_ctrl.nc', decode_times=False)
AMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/AMOC_rcp.nc' , decode_times=False)
PMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/PMOC_ctrl.nc', decode_times=False)
PMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/PMOC_rcp.nc' , decode_times=False)
GMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/GMOC_ctrl.nc', decode_times=False)
GMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/GMOC_rcp.nc' , decode_times=False)

In [ ]:
print(len(AMOC_ctrl.time), len(PMOC_ctrl.time), len(GMOC_ctrl.time))
print(len(AMOC_rcp .time), len(PMOC_rcp .time), len(GMOC_rcp .time))

In [ ]:
tdepth = create_tdepth('ocn')
lats = approx_lats('ocn')
X, Y = np.meshgrid(lats, -tdepth/1e3)

In [ ]:
cm_mean = discrete_cmap(16, 'RdBu_r')#cmocean.cm.balance)
cm_anom = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=0., midpoint=0.5, stop=.75, name='shrunk'))

In [ ]:
AMOC_ctrl

In [ ]:
approx_lats('ocn').plot()

In [ ]:
AMOC_ctrl.sel({'z_t':1e5}, method='nearest').mean(dim='time')

In [ ]:
np.argmax(AMOC_ctrl.sel({'z_t':1e5}, method='nearest').mean(dim='time'))

In [ ]:
approx_lats('ocn')[1561]

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(approx_lats('ocn'), AMOC_ctrl.sel({'z_t':1e5}, method='nearest').mean(dim='time')/1e6);
plt.scatter(approx_lats('ocn')[1561], AMOC_ctrl.sel({'z_t':1e5}, method='nearest').mean(dim='time')[1561]/1e6, c='r');
plt.axvline(26)
plt.axhline(0)
plt.tight_layout()

In [ ]:
# AMOC

f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, AMOC_ctrl.mean(dim='time')/1e6, vmin=-20, vmax=20, cmap=cm_mean)
plt.text(.06, .9, 'AMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='max')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_ctrl_mean')

    
f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, (AMOC_rcp.mean(dim='time')-AMOC_ctrl.mean(dim='time'))/1e6, vmin=-4, vmax=2, cmap=cm_anom)
plt.text(.06, .9, 'AMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$RCP$\rangle$-$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='min')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_rcp-ctrl')

In [ ]:
# PMOC

f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, PMOC_ctrl.mean(dim='time')/1e6, vmin=-20, vmax=20, cmap=cm_mean)
plt.text(.06, .9, 'PMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='max')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/PMOC_depth_lat_ctrl_mean')

    
f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, (PMOC_rcp.mean(dim='time')-PMOC_ctrl.mean(dim='time'))/1e6, vmin=-4, vmax=2, cmap=cm_anom)
plt.text(.06, .9, 'PMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$RCP$\rangle$-$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='min')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/PMOC_depth_lat_rcp-ctrl')

In [ ]:
# GMOC

f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, GMOC_ctrl.mean(dim='time')/1e6, vmin=-20, vmax=20, cmap=cm_mean)
plt.text(.06, .9, 'GMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='max')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/GMOC_depth_lat_ctrl_mean')

    
f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, (GMOC_rcp.mean(dim='time')-GMOC_ctrl.mean(dim='time'))/1e6, vmin=-4, vmax=2, cmap=cm_anom)
plt.text(.06, .9, 'GMOC', fontsize=16, transform=ax.transAxes)
plt.text(.05, .82, r'$\langle$RCP$\rangle$-$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='min')
cbar.ax.set_ylabel('volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/GMOC_depth_lat_rcp-ctrl')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(AMOC_ctrl.time/365     , AMOC_max(AMOC_ctrl)/1e6             , c='C0', lw=1)
plt.plot(AMOC_rcp .time/365-1800, AMOC_max(AMOC_rcp )/1e6             , c='C1', lw=1)
plt.plot(AMOC_ctrl.time/365     , lowpass(AMOC_max(AMOC_ctrl), 10)/1e6, c='C0', lw=3, label='CTRL')
plt.plot(AMOC_rcp .time/365-1800, lowpass(AMOC_max(AMOC_rcp ), 10)/1e6, c='C1', lw=3, label='RCP' )

plt.text(.98, .98, '26$^\circ$N, 1000 m', ha='right', va='top', fontsize=16, transform=ax.transAxes, color='grey')
plt.text(.6, .02, 'yearly data & 10 yr lowpass filtered', ha='center', va='bottom', fontsize=14, transform=ax.transAxes, color='grey')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('AMOC strength [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False, loc=3)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_timeseries')

### low resolution
`coord(dim)`

`transport_regions(transport_reg)`: 0 = Global, 1 = Atlantic

`moc_components(moc_comp)` 0 = "Eulerian Mean",  1 = "Eddy-Induced (bolus)", 2 = "Submeso" ;
  
transport_components =
  "Total",
  "Eulerian-Mean Advection",
  "Eddy-Induced Advection (bolus) + Diffusion",
  "Eddy-Induced (bolus) Advection",
  "Submeso Advection" ;

In [ ]:
from paths import CESM_filename

In [ ]:
print(file_ex_ocn_lpd)

In [ ]:
%%time
AMOC = xr.open_mfdataset(CESM_filename(run='lpi', domain='ocn_low', y=0, m=13), decode_times=False)\
                        .MOC.sel({'moc_z':1e5, 'lat_aux_grid':26})[:,1,1]
GMOC = xr.open_mfdataset(CESM_filename(run='lpd', domain='ocn_low', y=0, m=13), decode_times=False)\
                        .MOC.sel({'moc_z':1e5, 'lat_aux_grid':26})[:,0,1]

In [ ]:
AMOC.plot()

In [ ]:
ds.MOC

In [ ]:
ds.MOC.moc_comp

In [ ]:
for i in range(2):
    f, ax = plt.subplots(1,3, figsize=(15,5), sharey=True)
    for j in range(3):
        ds.MOC.sel({'time':0}, method='nearest')[i,j,:,:].plot(ax=ax[j])
    plt.tight_layout()

In [ ]:
ds.MOC.sel({'time':0, 'moc_z':1e5, 'lat_aux_grid':26}, method='nearest')[1,0]

In [ ]:
print(max(ds.MOC.sel({'time':0, 'moc_z':1e5}, method='nearest')[1,0,:]))
ds.MOC.sel({'time':0, 'moc_z':1e5}, method='nearest')[1,0,:].plot();

In [ ]:
np.argmax(ds.MOC.sel({'time':0, 'moc_z':1e5}, method='nearest')[1,0,:])

In [ ]:
ds.MOC.lat_aux_grid[290]

In [ ]:
ds.MOC.sel({'time':0}, method='nearest')[0,1,:,:].plot()

In [ ]:
ds.MOC.coords

In [ ]:
ds.MOC.dims

In [ ]:
for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='yrly', name='UVEL_VVEL')):

# Stability Criterion: Freshwater Flux at 35$^\circ$S